In [ ]:
"""
The code below downloads the CIFAR-10 dataset
"""

from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import tarfile

cifar10_dataset_folder_path = 'cifar-10-batches-py'
tar_gz_path = 'cifar-10-python.tar.gz'

class DLProgress(tqdm):#Progress Bar
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(tar_gz_path):#Download dataset
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='CIFAR-10 Dataset') as pbar:
        urlretrieve(
            'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz',
            tar_gz_path,
            pbar.hook)

if not isdir(cifar10_dataset_folder_path):
    with tarfile.open(tar_gz_path) as tar:
        tar.extractall()
        tar.close()

This dataset consist of 10 objects in 60000 32x32 images
The 10 objects are:
* Airplane
* Automobile
* Bird
* Cat
* Deer
* Dog
* Frog
* Horse
* Ship
* Truck

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as ply
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing

## Color doesn't matter when trying to classify images.

![Which is easier to classify](img/color.PNG)

Likewise neural networks don't know how to classify names, it cannot tell whether a picture is an airplane or a dog.
Instead we are going to transform them into vectors which look like this

![Labels -> Vectors](img/one-hot.PNG)

## How do we calculate how far off we were? What is our Error?

![Categorical Cross Entropy](img/cross-entropy.PNG)

## How do we Learn?

![Gradient Descent](img/GD.PNG)

Using gradient descent we can minimize our error, but for images its not practical to calculate.

### Stocastic Gradient Descent

![Stocastic Gradient Descent](img/SGD.PNG)

Instead lets take a subset of our error and "guess" the right direction to minimize our error. Less calculations but a lot more iterations

In [ ]:
def normalize(images):
    """
    Turn a color image into grayscale
    Colors are 3 dimensional, ranging from 0-255 for RGB
    """
    return NotImplemented


def one_hot_encoding(labels):
    """
    Transform labels from word description
    to vectors
    """
    return NotImplemented

# Preprocessing our dataset

## We are going to remove the color from the images using our normalize function and than store the labels as vectors through one_hot_encoding

In [ ]:
import pickle

"""
Set up files for batching
"""

def batches(cifar10_dataset_folder_path, normalize, one_hot_encode):
    """
    Preprocess Training and Validation Data
    """
    n_batches = 5
    valid_features = []
    valid_labels = []

    for batch_i in range(1, n_batches + 1):
        features, labels = load_cfar10_batch(cifar10_dataset_folder_path, batch_i)
        validation_count = int(len(features) * 0.1)

        # Process and save a batch of training data
        preprocess_and_save(
            normalize,
            one_hot_encode,
            features[:-validation_count],
            labels[:-validation_count],
            'preprocess_batch_' + str(batch_i) + '.p')

        # Use a portion of training batch for validation
        valid_features.extend(features[-validation_count:])
        valid_labels.extend(labels[-validation_count:])

    # Preprocess and Save all validation data
    preprocess_and_save(
        normalize,
        one_hot_encode,
        np.array(valid_features),
        np.array(valid_labels),
        'preprocess_validation.p')

    with open(cifar10_dataset_folder_path + '/test_batch', mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')

    # load the test data
    test_features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    test_labels = batch['labels']

    # Preprocess and Save all test data
    preprocess_and_save(
        normalize,
        one_hot_encode,
        np.array(test_features),
        np.array(test_labels),
        'preprocess_test.p')
    
def preprocess_and_save(normalize, one_hot_encode, features, labels, filename):
    """
    Preprocess data and save it to file
    """
    features = normalize(features)
    labels = one_hot_encode(labels)

    pickle.dump((features, labels), open(filename, 'wb'))
    
def load_cfar10_batch(cifar10_dataset_folder_path, batch_id):
    """
    Load a batch of the dataset
    """
    with open(cifar10_dataset_folder_path + '/data_batch_' + str(batch_id), mode='rb') as file:
        batch = pickle.load(file, encoding='latin1')

    features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    labels = batch['labels']

    return features, labels    

# Batching

## With GPU programming we have limited space, sometimes datasets are to large to hold in memory

## To overcome this we batch our data into multiple training sessions

In [ ]:
#create the batches
batches(cifar10_dataset_folder_path, normalize, one_hot_encoding)

## Checkpoint

Now that we have our data save, we are gonna load it and train on it

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
import problem_unittests as tests
from sklearn import preprocessing
import pickle

def load_preprocess_training_batch(batch_id, batch_size):
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    filename = 'preprocess_batch_' + str(batch_id) + '.p'
    features, labels = pickle.load(open(filename, mode='rb'))

    # Return the training data in batches of size <batch_size> or less
    return batch_features_labels(features, labels, batch_size)

def batch_features_labels(features, labels, batch_size):
    """
    Split features and labels into batches
    """
    for start in range(0, len(features), batch_size):
        end = min(start + batch_size, len(features))
        yield features[start:end], labels[start:end]


# Load the Preprocessed Validation data
valid_features, valid_labels = pickle.load(open('preprocess_validation.p', mode='rb'))

# Tensorflow syntax
## Basic data strctures and variables
- tf.placeholder(tf.typeID, shape, 'name') : type placeholder. shape and name are optional/ No shape implies single variable
- tf.Variable(initial-value, 'name') : variable placeholder
- tf.zeros(shape, 'name') : Initialize a tensor of zeros
- tf.truncated_normal(shape, mean, stddev) : will generate numbers from a random normal distribution using a given mean and standard deviation
- tf.nn.conv2d(tensor, weights, stride, padding) : weights and tensor must be of matching dimension, stride is the amount of data which is read in, padding can either be same or valid. 
- tf.nn.max_pool(tensor, kernel size, stride, padding) : Performs max-pooling with given kernel size and stride
- tf.contrib.layers.flatten(tensor) : Flattens a 4-D tensor to a 2-D tensor
- tf.contrib.layers.fully_connected(tensor, size) : Creates a fully connected layer of a given size
- tf.nn.dropout(tensor, probability) : performs dropout on a tensor

When giving None to tensorflow, it means that we are specifying a dynamic size

# Initializing Input

## We need to create placeholders for the images, labels, and our probability to keep our data

# Dropout

### One problem common with supervised learning algorithm is overfitting. We can overcome this by using dropout

### when will randomly 'turn off' parts of the network, never allowing for the Neural Network to fully learn the images. We want it to learn the shapes that are associated with the objects

![Stocastic Gradient Descent](img/dropout.PNG)

## Creating our Placeholders

In [ ]:
def neural_net_image_input(image_shape):
    """
    Return a Tensor for a batch of image input
    : image_shape: Shape of the images
    : return: Tensor for image input.
    """
    # TODO: Implement Function
    return tf.placeholder(tf.float32, (None ,) + image_shape, 'x')


def neural_net_label_input(n_classes):
    """
    Return a Tensor for a batch of label input
    : n_classes: Number of classes
    : return: Tensor for label input.
    """
    # TODO: Implement Function
    
    return tf.placeholder(tf.float32, (None, n_classes) , 'y')


def neural_net_keep_prob_input():
    """
    Return a Tensor for keep probability
    : return: Tensor for keep probability.
    """
    return tf.placeholder(tf.float32, name='keep_prob')

tf.reset_default_graph()    

## Implementing Convolutions and Max pooling

In [ ]:
def conv2d(inputs, filter_size, ksize, kstrides, padding, activation):
    """
    Apply Convolution to a Tensor
    :param inputs: Tensorflow Tensor
    :param filter_size: How deep the filter "weights" are
    :param ksize: Size of the kernel
    :param kstrides: How fast we slide the kernel
    :param padding: 'SAME' or 'VALID'
    :param activation: RELU
    """
    weights = tf.Variable(tf.truncated_normal([None], mean=None, stddev=None))
    bias = tf.Variable(tf.zeros([None]))
    tensor = tf.nn.conv2d(inputs, weights, 
                 strides=[1] + list(kstrides) + [1], 
                 padding=padding)
    
    tensor = tf.nn.bias_add(tensor, bias)
    return activation(tensor)

def conv2d_maxpool(inputs, pool_size, strides):
    """
    Apply max pooling on the input tensor
    :param Inputs: TensorFlow Tensor
    :param pool_size: kernel size that we are sliding across
    :param strides: Sliding speed
    """
    # TODO: Implement Function
    return tf.nn.max_pool(inputs, ksize=[1] + list(pool_size) + [1], strides = [1] + list(strides) + [1], padding='SAME')


def output(inputs, units):
    """
    Apply a output layer to x_tensor using weight and bias
    : x_tensor: A 2-D tensor where the first dimension is batch size.
    : num_outputs: The number of output that the new tensor should be.
    : return: A 2-D tensor where the second dimension is num_outputs.
    """
    # TODO: Implement Function
    
    weight = tf.Variable(tf.truncated_normal([inputs.get_shape()[1].value, units], mean=0, stddev=0.01))
    bias = tf.Variable(tf.zeros([units]))
    
    out = tf.matmul(inputs, weight)
    out = tf.nn.bias_add(out, bias)
    return out

tf.reset_default_graph()    

# Implementing VGG-19

## first we are going to take a look at LeNet

![LeNet](img/LeNet.PNG)

The output of our last layer, using the output function, is known as a logit.

# Our learning function, also known as an activation function, is called Relu

## Relu : Rectified Linear Units

We use Relu over the sigmoid function for multiple reasons. The most important one for now is that Relu are significantly faster to calculate versus sigmoid

![relu](img/relu.PNG)

# Our neural network will predict a vector with 10 values

## We want these values to be a percentage, one function which can create a percentage out of values is known as softmax

![soft](img/softmax.PNG)

In [ ]:
def conv_net(x, keep_prob):
    conv_ksize = [None, None]
    conv_strides = [None, None]
    pool_ksize = [None, None]
    pool_strides = [None, None]
    
    
    return NotImplemented
    
tf.reset_default_graph()    

inputs = neural_net_image_input((32,32,3)) #size of the images
labels = neural_net_label_input(10) #amount of labels
keep_prob = neural_net_keep_prob_input() #probability for dropout

logits = conv_net(inputs, keep_prob)

#Saving variables for use later
logits = tf.identity(logits, name='logits')

#Cost and gradient decent
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

# Now we are going to train our neural network

## We are also going to print out how well we do with each iteration

In [ ]:
def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    """
    Optimize the session on a batch of images and labels
    : session: Current TensorFlow session
    : optimizer: TensorFlow optimizer function
    : keep_probability: keep probability
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    """
    # TODO: Implement Function
    session.run(optimizer, feed_dict = {inputs: feature_batch, labels: label_batch, keep_prob: keep_probability})

In [ ]:
def print_stats(session, feature_batch, label_batch, cost, accuracy):
    """
    Print information about loss and validation accuracy
    : session: Current TensorFlow session
    : feature_batch: Batch of Numpy image data
    : label_batch: Batch of Numpy label data
    : cost: TensorFlow cost function
    : accuracy: TensorFlow accuracy function
    """
    # TODO: Implement Function
    loss = session.run(cost, feed_dict={inputs: feature_batch, labels: label_batch, keep_prob: 1})
    acc = session.run(accuracy, feed_dict={inputs: valid_features, labels: valid_labels, keep_prob: 1})
    
    print ('Loss: {:>10.4f} Validation: {:.6f}'.format(loss, acc))

## There are many parameters involved when building neural networks
- epochs : The amount of times we feed the entire dataset to the network
- batch size : How fast our network will learn. The better the GPU the larger this value can be. Traditionally in powers of 2
- Keep probability: How much of the network we want to randomly turn off at any given time

We are also going to save our model, we don't want to train our network each time we get new data. With tensorflow you can record the weights and biases you learned. 

In [ ]:
# Hyperparameters
epochs = 5
batch_size = 128
keep_probability = 0.8

save_model_path = './image_classification'

print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = 5
        for batch_i in range(1, n_batches + 1):
            for batch_features, batch_labels in load_preprocess_training_batch(batch_i, batch_size):
                train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
            print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
            print_stats(sess, batch_features, batch_labels, cost, accuracy)
            
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)
    sess.close()

# How well did we do?

## This was our accuracy against the training and validation set, now we will ask it to predict the label for images it has never seen before

In [ ]:
# Function to help us print our predictions
def display_image_predictions(features, labels, predictions):
    n_classes = 10
    label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    label_binarizer = LabelBinarizer()
    label_binarizer.fit(range(n_classes))
    label_ids = label_binarizer.inverse_transform(np.array(labels))

    fig, axies = plt.subplots(nrows=4, ncols=2)
    fig.tight_layout()
    fig.suptitle('Softmax Predictions', fontsize=20, y=1.1)

    n_predictions = 3
    margin = 0.05
    ind = np.arange(n_predictions)
    width = (1. - 2. * margin) / n_predictions

    for image_i, (feature, label_id, pred_indicies, pred_values) in enumerate(zip(features, label_ids, predictions.indices, predictions.values)):
        pred_names = [label_names[pred_i] for pred_i in pred_indicies]
        correct_name = label_names[label_id]

        axies[image_i][0].imshow(feature)
        axies[image_i][0].set_title(correct_name)
        axies[image_i][0].set_axis_off()

        axies[image_i][1].barh(ind + margin, pred_values[::-1], width)
        axies[image_i][1].set_yticks(ind + margin)
        axies[image_i][1].set_yticklabels(pred_names[::-1])
        axies[image_i][1].set_xticks([0, 0.5, 1.0])

We are going to import the model we just made and use the testing set to see how well we did

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf
import pickle
import helper
import random

# Set batch size if not already set
try:
    if batch_size:
        pass
except NameError:
    batch_size = 64

save_model_path = './image_classification'
n_samples = 4
top_n_predictions = 3

def test_model():
    """
    Test the saved model against the test dataset
    """

    test_features, test_labels = pickle.load(open('preprocess_test.p', mode='rb'))
    loaded_graph = tf.Graph()

    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        # Get Tensors from loaded model
        loaded_x = loaded_graph.get_tensor_by_name('x:0')
        loaded_y = loaded_graph.get_tensor_by_name('y:0')
        loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
        loaded_logits = loaded_graph.get_tensor_by_name('logits:0')
        loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')
        
        # Get accuracy in batches for memory limitations
        test_batch_acc_total = 0
        test_batch_count = 0
        
        for test_feature_batch, test_label_batch in batch_features_labels(test_features, test_labels, batch_size):
            test_batch_acc_total += sess.run(
                loaded_acc,
                feed_dict={loaded_x: test_feature_batch, loaded_y: test_label_batch, loaded_keep_prob: 1.0})
            test_batch_count += 1

        print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))

        # Print Random Samples
        random_test_features, random_test_labels = tuple(zip(*random.sample(list(zip(test_features, test_labels)), n_samples)))
        random_test_predictions = sess.run(
            tf.nn.top_k(tf.nn.softmax(loaded_logits), top_n_predictions),
            feed_dict={loaded_x: random_test_features, loaded_y: random_test_labels, loaded_keep_prob: 1.0})
        display_image_predictions(random_test_features, random_test_labels, random_test_predictions)


test_model()